In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# set eager execution true

tf.enable_eager_execution()
tfe = tf.contrib.eager

In [4]:
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [32]:
# Hyperparameters
lr = 0.1
bs = 100
num_steps = 1000
display_step = 50



In [8]:
# Iterator for the dataset

dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images, mnist.train.labels))
dataset = dataset.repeat().batch(bs).prefetch(bs)
dataset_iter = tfe.Iterator(dataset)

In [33]:
# Variables 
W = tfe.Variable(tf.zeros([784, 10]), name='Weights')
b = tfe.Variable(tf.zeros([10]), name='bias')

# Logistic regression = logit(W*x + b)

def logistic_regression(inputs):
    return tf.matmul(inputs, W) + b

def loss_fn(model_fn, inputs, labels):
    return tf.reduce_mean(\
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_fn(inputs), labels=labels))

def accuracy(model_fn, inputs, labels):
    preds = tf.nn.softmax(model_fn(inputs))
    correc_preds = tf.equal(tf.argmax(preds, 1), labels)
    return tf.reduce_mean(tf.cast(correc_preds, tf.float32))

In [34]:
## Optimizer 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)

## Gradients
grad = tfe.implicit_gradients(loss_fn)

In [35]:
# Training 

avg_loss = 0.0
avg_acc = 0.0

for step in range(num_steps):
    # iterate through dataset to get next batch
    d = dataset_iter.next()
    
    batch_xs = d[0]; batch_ys = tf.cast(d[1], dtype=tf.int64)
    
    # Compute the batch loss
    batch_loss = loss_fn(logistic_regression, batch_xs, batch_ys)
    avg_loss = batch_loss
    
    # computer the batch accuracy
    batch_acc = accuracy(logistic_regression, batch_xs, batch_ys)
    avg_acc = batch_acc
    
    if(step==0):
        print("Initial Loss: ", avg_loss.numpy())
    
    
    ## Update the variables with the gradients
    optimizer.apply_gradients(grad(logistic_regression, batch_xs, batch_ys))
    
    if(step+1)%display_step==0:
        print("Step: ", step+1, "loss: ", avg_loss.numpy(), "acc: ", avg_acc.numpy())
        avg_loss = 0.0
        avg_acc = 0.0

Initial Loss:  2.3025854
Step:  50 loss:  0.8161381 acc:  0.79
Step:  100 loss:  0.7015484 acc:  0.86
Step:  150 loss:  0.87807983 acc:  0.75
Step:  200 loss:  0.5499836 acc:  0.88
Step:  250 loss:  0.33186835 acc:  0.93
Step:  300 loss:  0.3777331 acc:  0.87
Step:  350 loss:  0.45444298 acc:  0.85
Step:  400 loss:  0.52011335 acc:  0.85
Step:  450 loss:  0.59544486 acc:  0.84
Step:  500 loss:  0.38248003 acc:  0.89
Step:  550 loss:  0.4988977 acc:  0.85
Step:  600 loss:  0.40463015 acc:  0.87
Step:  650 loss:  0.43248162 acc:  0.91
Step:  700 loss:  0.64552605 acc:  0.82
Step:  750 loss:  0.39121315 acc:  0.92
Step:  800 loss:  0.23755313 acc:  0.93
Step:  850 loss:  0.27960163 acc:  0.92
Step:  900 loss:  0.3496592 acc:  0.88
Step:  950 loss:  0.4392328 acc:  0.85
Step:  1000 loss:  0.5319768 acc:  0.86


In [36]:
## Evaluate model on the test image set
testX = mnist.test.images
testY = mnist.test.labels

test_acc = accuracy(logistic_regression, testX, testY)
print("Testset Accuracy: ", test_acc.numpy())

Testset Accuracy:  0.91
